In [4]:
%load_ext autoreload
%autoreload 2
%pylab inline

from collections import defaultdict, OrderedDict, Counter
from copy import deepcopy
import time
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.stats import poisson
from datetime import date, datetime
from common.databases.mdb import Reader
import uuid
import os
import ujson
import requests
from PIL import Image
import io
import tensorflow as tf
import pickle

from statoil import cfg
from statoil.utils import json_load, print_info, pickle_load


Populating the interactive namespace from numpy and matplotlib


# See the data

In [5]:
traindev = pickle_load(cfg.DATASETS["traindev_short"])

In [6]:
example = traindev[0]

for key in example.keys():
    print_info(example[key], key)

is_iceberg: val:0, type:<class 'int'>
id: val:5801c349, type:<class 'str'>
inc_angle: val:41.7067, type:<class 'float'>
band_1: type:<class 'numpy.ndarray'>, shape:(75, 75), dtype:float32, min:-33.031917572021484, max:10.201019287109375
synthetic: val:0, type:<class 'int'>
synthetic_method: val:None, type:<class 'NoneType'>
band_2: type:<class 'numpy.ndarray'>, shape:(75, 75), dtype:float32, min:-38.406253814697266, max:2.121691942214966


# Other checks

## Check dataset distint

In [8]:
train = pickle_load(cfg.DATASETS["train"])
dev = pickle_load(cfg.DATASETS["dev"])
traindev = pickle_load(cfg.DATASETS["traindev"])

train_ids = {example["id"] for example in train}
dev_ids = {example["id"] for example in dev}
traindev_ids = {example["id"] for example in traindev}

In [9]:
def print_data_stats(dataset, name):
    synthetic = sum([example.get("synthetic", 0) for example in dataset])
    print("{}: len:{}, orig_examples: {}({:.1f}%), synthetic: {}({:.1f}%)".format(
            name, 
            len(dataset), 
            len(dataset) - synthetic, 
            (len(dataset) - synthetic)/len(dataset)*100,
            synthetic, 
            synthetic/len(dataset)*100,
        ))

print_data_stats(train, "train")
print_data_stats(dev, "dev")
print_data_stats(traindev, "traindev")

train: len:20528, orig_examples: 1283(6.2%), synthetic: 19245(93.8%)
dev: len:5136, orig_examples: 321(6.2%), synthetic: 4815(93.8%)
traindev: len:25664, orig_examples: 1604(6.2%), synthetic: 24060(93.8%)


In [5]:

print("len(dev) = {}".format(len(dev)))
print("len(traindev) = {}".format(len(traindev)))

print(len(train_ids & dev_ids))
print(len(traindev_ids & dev_ids))
print(len(traindev_ids & train_ids))



len(train) = 23094
len(dev) = 321
len(traindev) = 1604
0
321
1283


### Compare loading times

In [26]:
%time traindev = json_load(cfg.RAW_TRAINSET)
%time test = json_load(cfg.RAW_TESTSET)

%time traindev = pickle_load(cfg.DATASETS["traindev"])
%time test = pickle_load(cfg.DATASETS["test"])

/home/prinda/.pyenv/versions/3.5.2/envs/py3/lib/python3.5/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


CPU times: user 1.26 s, sys: 228 ms, total: 1.48 s
Wall time: 1.98 s
CPU times: user 7.16 s, sys: 7.55 s, total: 14.7 s
Wall time: 17.6 s
CPU times: user 168 ms, sys: 56 ms, total: 224 ms
Wall time: 389 ms
CPU times: user 1.56 s, sys: 168 ms, total: 1.73 s
Wall time: 1.73 s


In [13]:
17/18


0.9444444444444444